## Random Forest Regressor - replicate

The notebook shows the recreated code of the person who took part in the competition.
At a later stage we will compare whether we managed to achieve better results with slightly changed or completely different approaches.

The results from the contest are slightly different from ours despite the same code. This could be caused by a newer version of the scikit-learn library.

Link to source code: https://www.kaggle.com/code/therealoise/bulldozer-price-prediction-randomforestregressor#2.-PROCESSING-DATA-FOR-MODELING

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
from dotenv import load_dotenv

while any(marker in os.getcwd() for marker in ('exercises', 'notebooks', 'students', 'research', 'projects')):
    os.chdir("..")
sys.path.append('src')

In [2]:
# Load environment variables from the correct .env file location
env_path = 'projects/proj_2_team_4/.env'
load_dotenv(env_path)

True

In [3]:
# Get dataset path from environment variable
train_and_valid_path = os.getenv('TRAIN_AND_VALID_PATH')

df = pd.read_csv(train_and_valid_path, low_memory=False, parse_dates=["saledate"])

###  LOADING THE TOOLS

In [4]:
#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Regular EDA and plotting libraries
import numpy as np

# Loading model
from sklearn.ensemble import RandomForestRegressor

# Loading model evaluators
from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

### DATA PREPROCESSING

##### PARSE DATES
##### To transform our saledate column into datetime data type, we will use the parse_dates parameter to turn it into real datetime type. So, we'll load the data again but this time, we'll parse dates.

In [6]:
# Let's confirm the changes
df.saledate.head(10)

0   2006-11-16
1   2004-03-26
2   2004-02-26
3   2011-05-19
4   2009-07-23
5   2008-12-18
6   2004-08-26
7   2005-11-17
8   2009-08-27
9   2007-08-09
Name: saledate, dtype: datetime64[ns]

##### SORT DATAFRAME BY SALEDATE

In [7]:
df.sort_values(by=["saledate"], inplace=True, ascending=True)
df.saledate.head(10)

205615   1989-01-17
274835   1989-01-31
141296   1989-01-31
212552   1989-01-31
62755    1989-01-31
54653    1989-01-31
81383    1989-01-31
204924   1989-01-31
135376   1989-01-31
113390   1989-01-31
Name: saledate, dtype: datetime64[ns]

In [8]:
# Let's make a copy of our dataframe
# Incase we'll have need of it in the future

df_new = df.copy()
df_new.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
205615,1646770,9500.0,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
274835,1821514,14000.0,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
141296,1505138,50000.0,1473654,4139,132,99.0,1978,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
212552,1671174,16000.0,1327630,8591,132,99.0,1980,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
62755,1329056,22000.0,1336053,4089,132,99.0,1984,NaN,NaN,1989-01-31,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,Lever,NaN,NaN


##### CREATE NEW COLUMNS
##### Next, let's create new columns: saleyear, salemonth, saleday, saledayofweek, and saledayofyear columns from the saledate column, to enrich our dataframe.

In [9]:
# New colums
df_new["saleyear"] = df_new.saledate.dt.year
df_new["salemonth"] = df_new.saledate.dt.month
df_new["saleday"] = df_new.saledate.dt.day
df_new["saledayofweek"] = df_new.saledate.dt.dayofweek
df_new["saledayofyear"] = df_new.saledate.dt.dayofyear

# Let's confirm the changes
df_new.head().T

,205615,274835,141296,212552,62755
SalesID,1646770,1821514,1505138,1671174,1329056
SalePrice,9500.0,14000.0,50000.0,16000.0,22000.0
MachineID,1126363,1194089,1473654,1327630,1336053
ModelID,8434,10150,4139,8591,4089
datasource,132,132,132,132,132
auctioneerID,18.0,99.0,99.0,99.0,99.0
YearMade,1974,1980,1978,1980,1984
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,1989-01-17 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00,1989-01-31 00:00:00


In [10]:
# Drop saledate
df_new.drop("saledate", axis=1, inplace=True)

##### CONVERT STRINGS TO CATEGORIES

In [11]:
for label, content in df_new.items():
    if pd.api.types.is_string_dtype(content):
        df_new[label] = content.astype("category").cat.as_ordered()
        
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412698 entries, 205615 to 409203
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   SalesID                   412698 non-null  int64   
 1   SalePrice                 412698 non-null  float64 
 2   MachineID                 412698 non-null  int64   
 3   ModelID                   412698 non-null  int64   
 4   datasource                412698 non-null  int64   
 5   auctioneerID              392562 non-null  float64 
 6   YearMade                  412698 non-null  int64   
 7   MachineHoursCurrentMeter  147504 non-null  float64 
 8   UsageBand                 73670 non-null   object  
 9   fiModelDesc               412698 non-null  category
 10  fiBaseModel               412698 non-null  category
 11  fiSecondaryDesc           271971 non-null  object  
 12  fiModelSeries             58667 non-null   object  
 13  fiModelDescriptor         748

##### FILL MISSING VALUES

In [12]:
for label, content in df_new.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

auctioneerID
MachineHoursCurrentMeter


In [13]:
for label, content in df_new.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # fill missing values with median
            df_new[label] = content.fillna(content.median())

In [14]:
print("Missing values in auctioneerID:", 
      df_new.auctioneerID.isna().sum())
print("Missing values in MachineHoursCurrentMeter:", 
      df_new.MachineHoursCurrentMeter.isna().sum())

Missing values in auctioneerID: 0
Missing values in MachineHoursCurrentMeter: 0


In [15]:
for label, content in df_new.items():
    if not pd.api.types.is_numeric_dtype(content):
        # Turn categories to numbers +1
        df_new[label] = pd.Categorical(content).codes+1

In [16]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412698 entries, 205615 to 409203
Data columns (total 57 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              412698 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  412698 non-null  float64
 8   UsageBand                 412698 non-null  int8   
 9   fiModelDesc               412698 non-null  int16  
 10  fiBaseModel               412698 non-null  int16  
 11  fiSecondaryDesc           412698 non-null  int16  
 12  fiModelSeries             412698 non-null  int8   
 13  fiModelDescriptor         412698 non-null  i

### MODELING

In [17]:
model = RandomForestRegressor(n_jobs=-1,
                             random_state=42)
# Fit the model
model.fit(df_new.drop("SalePrice", axis=1), df_new["SalePrice"])

RandomForestRegressor(n_jobs=-1, random_state=42)

In [18]:
# Split data into training and validation
df_val = df_new[df_new.saleyear == 2012]
df_train = df_new[df_new.saleyear != 2012]

# Check the length of both sets
len(df_val), len(df_train)

(11573, 401125)

In [19]:
# Split data into X and y
X_train, y_train = df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_valid, y_valid = df_val.drop("SalePrice", axis=1), df_val.SalePrice

# Check the shape of our X and y variables
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((401125, 56), (401125,), (11573, 56), (11573,))

In [20]:
# Create evaluation function (the competition uses Root Mean Square Log Error)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# Create function to evaluate our model
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_valid, y_valid)}
    return scores

In [21]:
# Let's change maximum samples to 30,000
model = RandomForestRegressor(n_jobs=-1,
                              max_samples=30000)

In [22]:
model.fit(X_train, y_train)

RandomForestRegressor(max_samples=30000, n_jobs=-1)

In [23]:
show_scores(model)

{'Training MAE': 4721.930005185414,
 'Valid MAE': 6549.926497018922,
 'Training RMSLE': np.float64(0.2237236232033258),
 'Valid RMSLE': np.float64(0.270751771147222),
 'Training R^2': 0.8971323518643166,
 'Valid R^2': 0.8591808977027062}

###  HYPERPARAMETER TUNING

In [24]:
# RandomForestRegressor hyperparameters
rf_grid = {"n_estimators": np.arange(10, 100, 50),
          "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2),
            "max_features": [0.5, 1, "sqrt", "auto"],
           "max_samples": [20000]}

# Tuning our model with RandomizedSearchCV
rs_rf = RandomizedSearchCV(RandomForestRegressor(),
                           param_distributions=rf_grid,
                           cv=5,
                           n_iter=20,
                           verbose=True)

# Fitting our X_train and y_train
rs_rf.fit(X_train, y_train);

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [25]:
rs_rf.best_params_

{'n_estimators': np.int64(10),
 'min_samples_split': np.int64(12),
 'min_samples_leaf': np.int64(1),
 'max_samples': 20000,
 'max_features': 'sqrt',
 'max_depth': None}

In [26]:
show_scores(rs_rf)

{'Training MAE': 6806.456471275296,
 'Valid MAE': 8556.71199223017,
 'Training RMSLE': np.float64(0.3035238918783594),
 'Valid RMSLE': np.float64(0.3500330365256642),
 'Training R^2': 0.7996276796437891,
 'Valid R^2': 0.7766234592902374}

In [27]:
# Best hyperparameters
best_rf = RandomForestRegressor(n_estimators=60,
                                    min_samples_leaf=13,
                                    min_samples_split=12,
                                    max_features='sqrt',
                                    n_jobs=-1,
                                    max_samples=None)
best_rf.fit(X_train, y_train)

RandomForestRegressor(max_features='sqrt', min_samples_leaf=13,
                      min_samples_split=12, n_estimators=60, n_jobs=-1)

In [28]:
show_scores(best_rf)

{'Training MAE': 5331.147348874749,
 'Valid MAE': 6938.095552383398,
 'Training RMSLE': np.float64(0.24385858775285324),
 'Valid RMSLE': np.float64(0.2832177793912945),
 'Training R^2': 0.8737253999215648,
 'Valid R^2': 0.843174998596525}